[![Open in GitHub](https://img.shields.io/badge/-Open%20in%20GitHub-black?logo=github)](https://github.com/Mearman/openalex-docs/blob/main/api-entities/works/search-works.ipynb)[![Open in Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/api-entities/works/search-works.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python.git"

In [ ]:
import json
from openalex_api import Configuration, ApiClient,AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

# Search works

The best way to search for works is to use the `search` query parameter, which searches across [titles](./work-object/README.md#title), [abstracts](./work-object/README.md#abstract\_inverted\_index), and [fulltext](./work-object/README.md#has_fulltext). Example:

* Get works with search term "dna" in the title, abstract, or fulltext:\
  [`https://api.openalex.org/works?search=dna`](https://api.openalex.org/works?search=dna)

In [ ]:
response = works_api.get_works(
	search="dna"
)

print(json.dumps(response.to_dict(), indent=2))

Fulltext search is available for a subset of works, see [`Work.has_fulltext`](./work-object/README.md#has_fulltext) for more details.

{% hint style="info" %}
You can read more about search [here](./../../how-to-use-the-api/get-lists-of-entities/search-entities.ipynb). It will show you how relevance score is calculated, how words are stemmed to improve search results, and how to do complex boolean searches.
{% endhint %}

In [ ]:
response = works_api.get_works(
	filter="title.search:cubist"
)

print(json.dumps(response.to_dict(), indent=2))

## Search a specific field

You can use search as a [filter](./../../how-to-use-the-api/get-lists-of-entities/filter-entity-lists.ipynb), allowing you to fine-tune the fields you're searching over. To do this, you append `.search` to the end of the property you are filtering for:

* Get works with "cubist" in the title:\
  [`https://api.openalex.org/works?filter=title.search:cubist`](https://api.openalex.org/works?filter=title.search:cubist)

The following fields can be searched within works:

| Search filter                                                                        | Field that is searched                                                  |
| ------------------------------------------------------------------------------------ | ----------------------------------------------------------------------- |
| [`abstract.search`](./filter-works.md#abstract.search)                                 | [`abstract_inverted_index`](./work-object/README.md#abstract\_inverted\_index)     |
| [`display_name.search`](./filter-works.md#display\_name.search-alias-title.search)     | [`display_name`](./work-object/README.md#display\_name)                            |
| [`fulltext.search`](./filter-works.md#fulltext.search)                                 | fulltext via [`n-grams`](./get-n-grams.ipynb)                                |
| [`title.search`](./filter-works.md#display\_name.search-alias-title.search)            | [`display_name`](./work-object/README.md#display\_name)                            |
| [`title_and_abstract.search`](./filter-works.md#title_and_abstractsearch)            | [`display_name`](./work-object/README.md#display\_name) and [`abstract_inverted_index`](./work-object/README.md#abstract\_inverted\_index)  |

You can also use the filter `default.search`, which works the same as using the [`search` parameter](./README.md#search-works).

### Why can't I search by name of related entity (author name, institution name, etc.)?

Rather than searching for the _names_ of entities related to works—such as authors, institutions, and sources—you need to search by a more unique identifier for that entity, like the OpenAlex ID. This means that there is a 2 step process:

1. Find the ID of the related entity. For example, if you're interested in works associated with NYU, you could search the `/institutions` endpoint for that name: [`https://api.openalex.org/institutions?search=nyu`](https://api.openalex.org/institutions?search=nyu). Looking at the first result, you'll see that the OpenAlex ID for NYU is `I57206974`.

In [ ]:
response = institutions_api.get_institutions(
	search="nyu"
)

print(json.dumps(response.to_dict(), indent=2))

2. Use a [filter ](./filter-works.ipynb)with the `/works` endpoint to get all of the works: [`https://api.openalex.org/works?filter=institutions.id:I57206974`](https://api.openalex.org/works?filter=institutions.id:I57206974).

In [ ]:
response = works_api.get_works(
	filter="institutions.id:I57206974"
)

print(json.dumps(response.to_dict(), indent=2))

Why can't you do this in just one step? Well, if you use the search term, "NYU," you might end up missing the ones that use the full name "New York University," rather than the initials. Sure, you could try to think of all possible variants and search for all of them, but you might miss some, and you risk putting in search terms that let in works that you're not interested in. Figuring out which works are actually associated with the "NYU" you're interested shouldn't be your responsibility—that's our job! We've done that work for you, so all the relevant works should be associated with one unique ID.

## Autocomplete works

You can autocomplete works to create a very fast type-ahead style search function:

* Autocomplete works with "tigers" in the title:\
  [`https://api.openalex.org/autocomplete/works?q=tigers`](https://api.openalex.org/autocomplete/works?q=tigers)

In [ ]:
response = autocomplete_api.get_autocomplet(
	q="tigers",
	id="works"
)

print(json.dumps(response.to_dict(), indent=2))

This returns a list of works titles with the author of each work set as the hint:

<pre class="language-json"><code class="lang-json">{ 
  "results": [
    {
      "id": "https://openalex.org/W2125098916",
      "display_name": "Crouching tigers, hidden prey: Sumatran tiger and prey populations in a tropical forest landscape",
      "hint": "Timothy G. O'Brien, Margaret F. Kinnaird, Hariyo T. Wibisono",
      "cited_by_count": 620,
      "works_count": null,
      "entity_type": "work",
      "external_id": "https://doi.org/10.1017/s1367943003003172"
    },
    ...
<strong>  ]
</strong><strong>}
</strong></code></pre>

{% hint style="info" %}
Read more about [autocomplete](./../../how-to-use-the-api/get-lists-of-entities/autocomplete-entities.ipynb).
{% endhint %}